# Imports

In [1]:
import os
import numpy as np
import subprocess
import pandas as pd
import requests
from urllib.parse import urljoin
import sqlite3
import json
import time
from datetime import datetime, timedelta
import re
from google.colab import drive
import random
from bs4 import BeautifulSoup
import csv
from urllib.parse import urlparse
from scipy.stats import f_oneway
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [2]:
# mounting google drive
drive.mount('/content/gdrive/', force_remount=True)
os.chdir("/content/gdrive/Shareddrives/ECS 260/final")

Mounted at /content/gdrive/


# Function Definitions

In [3]:
def add_column_if_not_exists(cursor, table_name, column_name, column_definition):
  # check if the column already exists
  cursor.execute(f"PRAGMA table_info({table_name});")
  existing_columns = [column[1] for column in cursor.fetchall()]

  if column_name not in existing_columns:
    # add the column if it does not exist
    cursor.execute(f"ALTER TABLE {table_name} ADD COLUMN {column_name} {column_definition};")

In [4]:
# putting the value in the range from 0 to 100
def normalize_column(column):
  min_val = column.min()
  max_val = column.max()
  return (column - min_val) / (max_val - min_val) * 100

In [5]:
def calculate_weighted_average(df):
  weights = {
      'health_score': 25,
      'security_numeric': 20,
      'popularity_numeric': 10,
      'rank': 10,
      'maintenance_numeric': 15,
      'community_numeric': 15,
      'lagging_dependencies_count': 5
  }
  values = [df[key] for key in weights.keys()]
  weights = list(weights.values())

  weight_sum = 0
  value_sum = 0
  for i in range(len(df) - 1):
    if np.isnan(values[i]):
      continue
    weight_sum += weights[i]
    value_sum += values[i] * weights[i]

  if value_sum == 0:
    return np.nan

  return value_sum/weight_sum

In [6]:
# categorize a metric based on threshold into 'HIGH', 'MID', 'LOW'
def categorize_metric(metric_name, conn, categorization_logic='soft'):
  """
  Soft Categorization:
    Categorizes the specified metric column of a DataFrame into 'HIGH', 'MID', or 'LOW' based on thresholds.

  Hard Categorization:
    Categorizes the specified metric column of a DataFrame without specifying any thresholds and copying the values as is.

  Parameters:
    metric_name (str): Name of the metric column to be categorized.
    conn (sqlite3.Connection): SQLite database connection object.
    categorization_logic: <'soft', 'hard'>

  Returns:
    DataFrame: DataFrame with a new column containing the assigned labels.
  """

  query = f"SELECT package, {metric_name} FROM package_metrics WHERE {metric_name} IS NOT NULL"
  package_data_df = pd.read_sql_query(query, conn)

  if categorization_logic == 'hard':
    package_data_df[f'{metric_name}'] = package_data_df[metric_name]
    return package_data_df

  # Convert the specified metric column to numeric, coercing errors to NaN
  package_data_df[metric_name] = pd.to_numeric(package_data_df[metric_name], errors='coerce')

  # Calculate the range of the specified metric
  min_value = package_data_df[metric_name].min()
  max_value = package_data_df[metric_name].max()
  value_range = max_value - min_value

  # Calculate thresholds for categorizing the specified metric
  low_threshold = min_value + (value_range / 3)
  high_threshold = min_value + (2 * value_range / 3)

  # Function to assign label based on thresholds
  def assign_label(value):
    if pd.isna(value):
      return None
    elif value > high_threshold:
      return 'HIGH'
    elif low_threshold <= value <= high_threshold:
      return 'MID'
    else:
      return 'LOW'

  # Create a new column containing the assigned labels
  package_data_df[f'{metric_name}'] = package_data_df[metric_name].apply(assign_label)

  # Return the DataFrame with only the newly created column
  return package_data_df

In [7]:
# update database
def update_metric_categorization(df, table_name, column_name, identifier_column, conn, cursor):
  values_to_update = [(row[column_name], row[identifier_column]) for _, row in df.iterrows()]
  update_query = f"UPDATE {table_name} SET {column_name} = ? WHERE {identifier_column} = ?"
  cursor.executemany(update_query, values_to_update)
  conn.commit()

In [8]:
def check_if_state(df_val, state):
  if df_val == None:
    return False
  df_val = json.loads(df_val)
  return state in df_val

# DB Init

In [9]:
# init database and cursor
conn = sqlite3.connect(f"final_database.db")
cursor = conn.cursor()

In [10]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS metric_analysis (
        package TEXT
    )
""")

conn.commit()

In [11]:
# getting final metrics to be used for RQ3 security analysis
with open('metrics_for_security.txt', "r") as file:
  metrics_for_security = file.read()

metrics_for_security = json.loads(metrics_for_security)
n_cols = len(metrics_for_security)
print(f"{n_cols} metrics to be used for RQ3 security:\n{metrics_for_security}")

add_column_if_not_exists(cursor, 'metric_analysis', 'threat_score', 'TEXT DEFAULT NULL')
conn.commit()

7 metrics to be used for RQ3 security:
['lagging_dependencies_count', 'health_score', 'security_numeric', 'popularity_numeric', 'maintenance_numeric', 'community_numeric', 'rank']


In [12]:
sql_query = """
    INSERT INTO metric_analysis (package)
    SELECT p.package
    FROM package_data p
    LEFT JOIN metric_analysis m ON p.package = m.package
    WHERE m.package IS NULL
"""
cursor.execute(sql_query)
conn.commit()

In [13]:
df = pd.read_sql_query(f"SELECT * FROM metric_analysis", conn)
df

,package,is_deprecated,dependants_count,forks,issues,pr,contributors,unpack_size,total_lines_of_code,sloc,days_since_last_modification,avg_commit_freq,threat_score,file_count,state
0,@gerrico/react-components,0,LOW,LOW,LOW,LOW,LOW,MID,MID,MID,LOW,MID,LOW,MID,"[""DEAD"", ""TRIVIAL""]"
1,express-simple-app-generator,0,LOW,None,None,None,None,HIGH,None,None,LOW,None,MID,None,"[""TRIVIAL"", ""ACTIVE""]"
2,generator-giraffe,0,LOW,LOW,LOW,LOW,LOW,None,MID,MID,LOW,MID,LOW,None,"[""DEAD"", ""TRIVIAL""]"
3,outdated-client,0,LOW,None,None,None,None,MID,None,None,LOW,None,LOW,MID,"[""ACTIVE""]"
4,@semi-bot/semi-theme-shopify,0,None,None,None,None,None,MID,None,None,LOW,None,MID,MID,"[""ACTIVE""]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,haribotify,0,LOW,None,None,None,None,None,None,None,LOW,None,MID,None,"[""TRIVIAL"", ""ACTIVE""]"
29996,eslint-config-sharecar,0,LOW,LOW,LOW,LOW,LOW,MID,MID,LOW,LOW,LOW,MID,LOW,"[""DEAD"", ""TRIVIAL""]"
29997,webpack-to-ardoq,0,LOW,None,None,None,None,MID,None,None,LOW,None,LOW,MID,"[""ACTIVE""]"
29998,zywave-content-search,0,LOW,None,None,None,None,MID,None,None,LOW,None,LOW,MID,"[""ACTIVE""]"


# Updating threat score

In [ ]:
add_column_if_not_exists(cursor, 'package_metrics', 'threat_score', 'TEXT DEFAULT NULL')
conn.commit()

In [ ]:
table_name = 'package_metrics'
identifier_column = 'package'
df = pd.read_sql_query(f'SELECT {identifier_column}, {", ".join(metrics_for_security)} FROM {table_name}', conn)
for metric in metrics_for_security:
  df[metric] = pd.to_numeric(df[metric], errors='coerce')
  if metric in ['rank', 'lagging_dependencies_count']:
    df[metric] = normalize_column(df[metric])
df['threat_score'] = df.apply(calculate_weighted_average, axis=1)
update_metric_categorization(df, table_name, 'threat_score', identifier_column, conn, cursor)

In [ ]:
pd.to_numeric(df['threat_score'], errors='coerce').describe()

count    29365.000000
mean        38.382001
std          9.490004
min          0.459770
25%         30.189552
50%         39.280295
75%         41.712303
max         95.271097
Name: threat_score, dtype: float64

In [ ]:
table_name = 'metric_analysis'
identifier_column = 'package'
for metric in ['threat_score']:
  print('----------------------------------------------------------------------------------------------------------------------------')
  print(f"Now updating {metric} ...")
  update_metric_categorization(categorize_metric(metric, conn), table_name, metric, identifier_column, conn, cursor)
  print(f"Updated {metric}")
  print('----------------------------------------------------------------------------------------------------------------------------')
for metric in ['threat_score']:
  cursor.execute(f"SELECT DISTINCT {metric} FROM {table_name}")
  print(f"Metric {metric} --> {[row[0] for row in cursor.fetchall()]}")

----------------------------------------------------------------------------------------------------------------------------
Now updating threat_score ...
Updated threat_score
----------------------------------------------------------------------------------------------------------------------------
Metric threat_score --> ['LOW', 'MID', None, 'HIGH']


# RQ3 Analysis - performing ANOVA test

In [ ]:
df = pd.read_sql_query("SELECT pm.*, ma.state FROM package_metrics as pm JOIN metric_analysis as ma ON pm.package = ma.package", conn)
active_threat = df[df.apply(lambda row: check_if_state(row['state'], 'ACTIVE'), axis=1)]['threat_score']
deprecated_threat = df[df.apply(lambda row: check_if_state(row['state'], 'DEPRECATED'), axis=1)]['threat_score']
dead_threat = df[df.apply(lambda row: check_if_state(row['state'], 'DEAD'), axis=1)]['threat_score']
trivial_threat = df[df.apply(lambda row: check_if_state(row['state'], 'TRIVIAL'), axis=1)]['threat_score']

In [ ]:
# check for missing values (None or NaN) and count non-missing values in 'active_threat'
active_threat_missing = active_threat.isna().sum()
active_threat_count = active_threat.notna().sum()

# check for missing values (None or NaN) and count non-missing values in 'deprecated_threat'
deprecated_threat_missing = deprecated_threat.isna().sum()
deprecated_threat_count = deprecated_threat.notna().sum()

# check for missing values (None or NaN) and count non-missing values in 'dead_threat'
dead_threat_missing = dead_threat.isna().sum()
dead_threat_count = dead_threat.notna().sum()

# check for missing values (None or NaN) and count non-missing values in 'trivial_threat'
trivial_threat_missing = trivial_threat.isna().sum()
trivial_threat_count = trivial_threat.notna().sum()

# print results
print("Missing values (None or NaN) and non-missing value counts:")
print("==> Active Threat:\tMissing --> ", active_threat_missing, "\tNon-missing --> ", active_threat_count)
print("==> Deprecated Threat:\tMissing --> ", deprecated_threat_missing, "\tNon-missing --> ", deprecated_threat_count)
print("==> Dead Threat:\tMissing --> ", dead_threat_missing, "\tNon-missing --> ", dead_threat_count)
print("==> Trivial Threat:\tMissing --> ", trivial_threat_missing, "\tNon-missing --> ", trivial_threat_count)

Missing values (None or NaN) and non-missing value counts:
==> Active Threat:	Missing -->  440 	Non-missing -->  18235
==> Deprecated Threat:	Missing -->  0 	Non-missing -->  1025
==> Dead Threat:	Missing -->  2 	Non-missing -->  11103
==> Trivial Threat:	Missing -->  92 	Non-missing -->  19789


In [ ]:
# drop NaN values and ensure uniform length
active_threat_cleaned = active_threat.dropna().sample(min(len(active_threat.dropna()), len(deprecated_threat.dropna()), len(dead_threat.dropna()), len(trivial_threat.dropna())))
deprecated_threat_cleaned = deprecated_threat.dropna().sample(min(len(active_threat.dropna()), len(deprecated_threat.dropna()), len(dead_threat.dropna()), len(trivial_threat.dropna())))
dead_threat_cleaned = dead_threat.dropna().sample(min(len(active_threat.dropna()), len(deprecated_threat.dropna()), len(dead_threat.dropna()), len(trivial_threat.dropna())))
trivial_threat_cleaned = trivial_threat.dropna().sample(min(len(active_threat.dropna()), len(deprecated_threat.dropna()), len(dead_threat.dropna()), len(trivial_threat.dropna())))

# Given statistics
non_missing_counts = {
    'active_threat': active_threat_count,
    'deprecated_threat': deprecated_threat_count,
    'dead_threat': dead_threat_count,
    'trivial_threat' : trivial_threat_count
}

# perform one-way ANOVA
f_statistic, p_value = f_oneway(active_threat_cleaned, deprecated_threat_cleaned, dead_threat_cleaned, trivial_threat_cleaned)

print("ANOVA Results:")
print("\tF-Statistic:", f_statistic)
print("\tP-Value:", p_value)
print()
if p_value < 0.05:
  print("==> The means of the groups are significantly different (Reject null hypothesis).")
else:
  print("==> There is no significant difference between the means of the groups (Fail to reject null hypothesis).")

ANOVA Results:
	F-Statistic: 9.043853068018349
	P-Value: 5.7597467432457056e-06

==> The means of the groups are significantly different (Reject null hypothesis).


In [ ]:
df = pd.read_sql_query("SELECT threat_score, state FROM metric_analysis", conn)
active_threat_type = df[df.apply(lambda row: check_if_state(row['state'], 'ACTIVE'), axis=1)]['threat_score']
deprecated_threat_type = df[df.apply(lambda row: check_if_state(row['state'], 'DEPRECATED'), axis=1)]['threat_score']
dead_threat_type = df[df.apply(lambda row: check_if_state(row['state'], 'DEAD'), axis=1)]['threat_score']
trivial_threat_type = df[df.apply(lambda row: check_if_state(row['state'], 'TRIVIAL'), axis=1)]['threat_score']

In [ ]:
state_counts = active_threat_type.value_counts()
# print("Threat score mean of different states:")
print("---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
print("ACTIVE:")
state_counts = active_threat_type.value_counts()
print("\t", "High n Mid --> ", state_counts[['HIGH', 'MID']].sum(), " | ", "{:.2f}".format(state_counts[['HIGH', 'MID']].sum()/state_counts.sum()*100), "%",
      "\tHigh --> ", state_counts[['HIGH']].sum(), " | ", "{:.2f}".format(state_counts[['HIGH']].sum()/state_counts.sum()*100), "%",
      "\tMid --> ", state_counts[['MID']].sum(), " | ", "{:.2f}".format(state_counts[['MID']].sum()/state_counts.sum()*100), "%",
      "\tLow --> ", state_counts[['LOW']].sum(), " | ", "{:.2f}".format(state_counts[['LOW']].sum()/state_counts.sum()*100), "%",
      "\t High to Low Ratio --> ", "{:.2f}".format(state_counts[['HIGH']].sum()/state_counts[['LOW']].sum()))
print("---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
print("TRIVIAL:")
state_counts = trivial_threat_type.value_counts()
print("\t", "High n Mid --> ", state_counts[['HIGH', 'MID']].sum(), " | ", "{:.2f}".format(state_counts[['HIGH', 'MID']].sum()/state_counts.sum()*100), "%",
      "\tHigh --> ", state_counts[['HIGH']].sum(), " | ", "{:.2f}".format(state_counts[['HIGH']].sum()/state_counts.sum()*100), "%",
      "\tMid --> ", state_counts[['MID']].sum(), " | ", "{:.2f}".format(state_counts[['MID']].sum()/state_counts.sum()*100), "%",
      "\tLow --> ", state_counts[['LOW']].sum(), " | ", "{:.2f}".format(state_counts[['LOW']].sum()/state_counts.sum()*100), "%",
      "\t High to Low Ratio --> ", "{:.2f}".format(state_counts[['HIGH']].sum()/state_counts[['LOW']].sum()))
print("---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
print("DEAD:")
state_counts = dead_threat_type.value_counts()
print("\t", "High n Mid --> ", state_counts[['HIGH', 'MID']].sum(), " | ", "{:.2f}".format(state_counts[['HIGH', 'MID']].sum()/state_counts.sum()*100), "%",
      "\tHigh --> ", state_counts[['HIGH']].sum(), " | ", "{:.2f}".format(state_counts[['HIGH']].sum()/state_counts.sum()*100), "%",
      "\tMid --> ", state_counts[['MID']].sum(), " | ", "{:.2f}".format(state_counts[['MID']].sum()/state_counts.sum()*100), "%",
      "\tLow --> ", state_counts[['LOW']].sum(), " | ", "{:.2f}".format(state_counts[['LOW']].sum()/state_counts.sum()*100), "%",
      "\t High to Low Ratio --> ", "{:.2f}".format(state_counts[['HIGH']].sum()/state_counts[['LOW']].sum()))
print("---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
print("DEPRECATED:")
state_counts = deprecated_threat_type.value_counts()
print("\t", "High n Mid --> ", state_counts[['HIGH', 'MID']].sum(), " | ", "{:.2f}".format(state_counts[['HIGH', 'MID']].sum()/state_counts.sum()*100), "%",
      "\tHigh --> ", state_counts[['HIGH']].sum(), " | ", "{:.2f}".format(state_counts[['HIGH']].sum()/state_counts.sum()*100), "%",
      "\t\tMid --> ", state_counts[['MID']].sum(), " | ", "{:.2f}".format(state_counts[['MID']].sum()/state_counts.sum()*100), "%",
      "\tLow --> ", state_counts[['LOW']].sum(), " | ", "{:.2f}".format(state_counts[['LOW']].sum()/state_counts.sum()*100), "%",
      "\t High to Low Ratio --> ", "{:.2f}".format(state_counts[['HIGH']].sum()/state_counts[['LOW']].sum()))
print("---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
ACTIVE:
	 High n Mid -->  12993  |  71.25 % 	High -->  360  |  1.97 % 	Mid -->  12633  |  69.28 % 	Low -->  5242  |  28.75 % 	 High to Low Ratio -->  0.07
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
TRIVIAL:
	 High n Mid -->  14349  |  72.51 % 	High -->  137  |  0.69 % 	Mid -->  14212  |  71.82 % 	Low -->  5440  |  27.49 % 	 High to Low Ratio -->  0.03
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
DEAD:
	 High n Mid -->  8095  |  72.91 % 	High -->  117  |  1.05 % 	Mid -->  7978  |  71.85 % 	Low -->  3008  |  27.09 % 	 High to Low Ratio -->  0.04
----

# Model Testing via Random Forest

In [ ]:
with open('metrics_for_categorization.txt', "r") as file:
  metrics_for_categorization = file.read()
metrics_for_categorization = json.loads(metrics_for_categorization)

columns_str = ', '.join([f"pm.{metric}" for metric in metrics_for_categorization])
sql_query = f"""
  SELECT pm.package, {columns_str}, ma.state
  FROM package_metrics AS pm
  JOIN metric_analysis AS ma ON pm.package = ma.package
"""

df = pd.read_sql_query(sql_query, conn)

def process_state(state, target_state):
  if state == None:
    return 0
  state_list = json.loads(state)
  if target_state in state_list:
    return 1
  else:
    return 0

for state in ['ACTIVE', 'DEAD', 'DEPRECATED', 'TRIVIAL']:
  df[state] = df['state'].apply(lambda x: process_state(x, state))

In [ ]:
def train_and_print_feature_importance(X, y, state_name):
    print(f"\n{'='*10} {state_name} {'='*10}")

    # Step 2: Drop rows with missing values
    X_dropped = X.dropna()
    y_dropped = y[X_dropped.index]  # Match indices after dropping rows

    # Step 3: Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_dropped, y_dropped, test_size=0.2, random_state=42)

    # Step 4: Select a classification model
    model = RandomForestClassifier()

    # Step 5: Train the model
    model.fit(X_train, y_train)

    # Step 6: Evaluate the model
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)

    # Step 7: Feature Importance Analysis
    feature_importances = model.feature_importances_
    sorted_indices = np.argsort(feature_importances)[::-1]

    # Print feature importance scores
    print("Feature Importance Scores:")
    for idx in sorted_indices:
        print(f"Feature '{feature_columns[idx]}': {feature_importances[idx]}")

In [ ]:
# Step 1: Select features (X) and target (y)
feature_columns = ['is_deprecated', 'dependants_count', 'forks', 'issues', 'pr', 'contributors',
                   'unpack_size', 'avg_commit_freq', 'total_lines_of_code', 'sloc',
                   'days_since_last_modification', 'file_count']

for state in ['ACTIVE', 'DEAD', 'DEPRECATED', 'TRIVIAL']:
    X = df[feature_columns]  # Selecting the specified feature columns
    y = df[state]  # Assuming state column contains the labeled state

    train_and_print_feature_importance(X, y, state)


========== ACTIVE ==========
Accuracy: 1.0
Feature Importance Scores:
Feature 'pr': 0.39696506849294566
Feature 'forks': 0.22147056318795688
Feature 'avg_commit_freq': 0.09609140054693924
Feature 'issues': 0.08090646686442167
Feature 'contributors': 0.0640694224597885
Feature 'is_deprecated': 0.03548133713165355
Feature 'dependants_count': 0.0322590530161417
Feature 'total_lines_of_code': 0.026927222936506574
Feature 'sloc': 0.015162534136241508
Feature 'unpack_size': 0.013536742827457955
Feature 'file_count': 0.008655643788881348
Feature 'days_since_last_modification': 0.008474544611065407

========== DEAD ==========
Accuracy: 1.0
Feature Importance Scores:
Feature 'pr': 0.4168828587234351
Feature 'forks': 0.20730109948560035
Feature 'avg_commit_freq': 0.09763393158267383
Feature 'contributors': 0.07209732552300853
Feature 'issues': 0.07015790056042397
Feature 'is_deprecated': 0.03517698431189517
Feature 'dependants_count': 0.028887611653764085
Feature 'total_lines_of_code': 0.023109